<a href="https://colab.research.google.com/github/wailokkwok/seq2seq_translator/blob/main/seq2seq_translation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('yue.txt') as f:
    lines = f.readlines()

df = lines

df1 = []
for x in df:
  df1.append(x.split("\t"))

In [ ]:
eng_ds = []
chin_ds = []

for data in df1:
  eng_sen = []
  eng_sen.append(data[0].split())
  eng_ds.append(eng_sen)

  chin_sen = []
  for char in data[1]:
    chin_sen.append(char)
  chin_sen.append("<end>")
  chin_sen.insert(0, "<start>")

  chin_ds.append(chin_sen)




















In [ ]:
print(chin_ds[0])

['<start>', '開', '始', '啦', '。', '<end>']


# Build dictionary for English and Chinese ds




In [ ]:
print(chin_ds[0][0])

<start>


In [ ]:
eng_dic = {}
chin_dic = {}

In [ ]:
x = 0
y = 0

for eng_sentence in eng_ds:
  for eng_char in eng_sentence[0]:
    if eng_char not in eng_dic.keys():
      eng_dic[eng_char] = x
      x += 1

for chin_sentence in chin_ds:
  for chin_char in chin_sentence:
    if chin_char not in chin_dic.keys():
      chin_dic[chin_char] = y
      y += 1




In [ ]:
ha = []
for text in eng_ds:
  ha.append(text[0])


max_encoder_seq_length = max([len(text) for text in ha])

In [ ]:
max_decoder_seq_length = max([len(text) for text in chin_ds])

# One hot encode to input, output and target

In [ ]:
import numpy as np

one_hot_eng = np.zeros((len(eng_ds), max_encoder_seq_length,len(eng_dic)))
one_hot_chin = np.zeros((len(chin_ds),max_decoder_seq_length,len(chin_dic)))
one_hot_chin_target = np.zeros((len(chin_ds),max_decoder_seq_length,len(chin_dic)))

In [ ]:
eng_ds[0][0]

['Begin.']

In [ ]:
for index, eng_sentence in enumerate(eng_ds):
  for eng_char in eng_sentence:
    for x_index, char in enumerate(eng_char):
      h = eng_dic[char]
      one_hot_eng[index][x_index][h] = 1




In [ ]:
for index, chin_sentence in enumerate(chin_ds):
  for x_index, char in enumerate(chin_sentence):
    h = chin_dic[char]
    one_hot_chin[index][x_index][h] = 1


In [ ]:
for index, chin_sentence in enumerate(chin_ds):
  for x_index, char in enumerate(chin_sentence):
    h = chin_dic[char]
    one_hot_chin_target[index][x_index-1][h] = 1

In [ ]:
print(one_hot_eng.shape, one_hot_chin.shape, one_hot_chin_target.shape)

(3293, 32, 4103) (3293, 47, 1580) (3293, 47, 1580)


In [ ]:
len(one_hot_chin[0][0])

1580

# Write Encoder

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

encoder_model = Input(shape = (None,len(one_hot_eng[0][0])))
_, h, c = LSTM(256, return_state = True)(encoder_model)
encoder_states = [h,c]

# Write Decoder

In [ ]:
decoder_model = Input(shape=(None, len(one_hot_chin[0][0])))
decoder_output = LSTM(256, return_sequences=True)(decoder_model,  initial_state = encoder_states)
decoder_softmax = Dense(len(one_hot_chin[0][0]), activation='softmax')(decoder_output)




# Encoder, Decoder

In [ ]:
model = Model ([encoder_model, decoder_model], decoder_softmax)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.fit(x=[one_hot_eng, one_hot_chin], y=one_hot_chin_target,
          batch_size = 64, epochs=100)

Epoch 1/100
52/52 [==============================] - 95s 2s/step - loss: 1.6269
Epoch 2/100
52/52 [==============================] - 84s 2s/step - loss: 1.4770
Epoch 3/100
52/52 [==============================] - 83s 2s/step - loss: 1.4761
Epoch 4/100
52/52 [==============================] - 86s 2s/step - loss: 1.4708
Epoch 5/100
52/52 [==============================] - 89s 2s/step - loss: 1.4939
Epoch 6/100
52/52 [==============================] - 93s 2s/step - loss: 1.4993
Epoch 7/100
52/52 [==============================] - 88s 2s/step - loss: 1.4879
Epoch 8/100
52/52 [==============================] - 87s 2s/step - loss: 1.4733
Epoch 9/100
52/52 [==============================] - 88s 2s/step - loss: 1.4642
Epoch 10/100
52/52 [==============================] - 89s 2s/step - loss: 1.4563
Epoch 11/100
52/52 [==============================] - 87s 2s/step - loss: 1.4501
Epoch 12/100
52/52 [==============================] - 88s 2s/step - loss: 1.4449
Epoch 13/100
52/52 [=================

## Prediction

In [ ]:
encoder_input = Model(inputs = encoder_model, outputs = encoder_states)

decoder_states_h = Input(shape = (256))
decoder_states_c = Input(shape = (256))
decoder_input_states = [decoder_states_h,decoder_states_c]


decoder_model = Input(shape=(None, len(one_hot_chin[0][0])))
decoder_output, h, c = LSTM(256, return_sequences=True, return_state = True)(decoder_model, initial_state = decoder_input_states)
decoder_output_states = [h,c]

decoder_softmax = Dense(len(one_hot_chin[0][0]), activation='softmax')(decoder_output)

decoder_output = Model (inputs = [decoder_model]+ decoder_input_states, outputs = [decoder_softmax]+ decoder_output_states)

In [ ]:
seed = "What a nice day"

one_hot_seed = np.zeros((1, 1,len(eng_dic)))

for index, x in enumerate(seed.split()):
  one_hot_seed[0][0][eng_dic[x]] = 1

In [ ]:
decoder_input_states = encoder_input.predict(one_hot_seed)


one_hot_output = np.zeros((1,1,len(chin_dic)))
one_hot_output[0][0][chin_dic["<start>"]] = 1








1/1 [==============================] - 0s 25ms/step


In [ ]:
one_hot_output.shape

(1, 1, 1580)

In [ ]:
line = ""
hex = True

output_token, h, c = decoder_output.predict([one_hot_output] + decoder_input_states)
decoder_input_states = [h,c]
answer = np.argmax(output_token)
one_hot_output = np.zeros((1,1,len(chin_dic)))
one_hot_output[0][0][answer] = 1

while answer != 5:
  output_token, h, c = decoder_output.predict([one_hot_output] + decoder_input_states)
  decoder_input_states = [h,c]
  answer = np.argmax(output_token)
  one_hot_output = np.zeros((1,1,len(chin_dic)))
  one_hot_output[0][0][answer] = 1

  line += list(chin_dic.keys())[list(chin_dic.values()).index(answer)]
  print(line)

1/1 [==============================] - 0s 25ms/step
棄
1/1 [==============================] - 0s 21ms/step
棄士
1/1 [==============================] - 0s 29ms/step
棄士士
1/1 [==============================] - 0s 24ms/step
棄士士食
1/1 [==============================] - 0s 27ms/step
棄士士食墨
1/1 [==============================] - 0s 23ms/step
棄士士食墨叭
1/1 [==============================] - 0s 31ms/step
棄士士食墨叭隊
1/1 [==============================] - 0s 23ms/step
棄士士食墨叭隊把
1/1 [==============================] - 0s 21ms/step
棄士士食墨叭隊把把
1/1 [==============================] - 0s 23ms/step
棄士士食墨叭隊把把把
1/1 [==============================] - 0s 30ms/step
棄士士食墨叭隊把把把把
1/1 [==============================] - 0s 31ms/step
棄士士食墨叭隊把把把把把
1/1 [==============================] - 0s 32ms/step
棄士士食墨叭隊把把把把把比
1/1 [==============================] - 0s 23ms/step
棄士士食墨叭隊把把把把把比噃
1/1 [==============================] - 0s 26ms/step
棄士士食墨叭隊把把把把把比噃噃
1/1 [==============================] - 0s 22ms/step
棄士士食墨叭隊把把把把把比噃噃藍
1/1 [===========

KeyboardInterrupt: ignored